#Packages

In [1]:
import tensorflow as tf
import os, fnmatch
import PIL
import numpy as np
import h5py
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.model_selection import train_test_split
import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from keras.layers import MaxPooling2D, Flatten, Dense, Dropout
from keras.initializers import GlorotUniform
from keras.regularizers import l2


#Import data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
tf.device('/gpu:0')
dir = "/content/drive/MyDrive/Image/Dataset1/"
test_dir = dir
label = []
list_of_folders = os.listdir(test_dir)
frame_paths = []

for folder in list_of_folders:
    print("folder: ", folder)
    list_of_files = os.listdir(test_dir + folder)
    pattern = "*.jpg"
    counter = 0
    for entry in list_of_files:
        if fnmatch.fnmatch(entry, pattern):
            file_path = test_dir + folder + "/" + entry
            frame_paths.append(file_path)
            label.append(folder)

folder:  Non-Smoke
folder:  Non-Smoke + Fog
folder:  Smoke + Fog
folder:  Smoke


In [8]:
train_val_data, test_data, train_val_labels, test_labels = train_test_split(frame_paths, label, test_size=0.5, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_val_data, train_val_labels, test_size=0.3, random_state=42)

In [9]:
tf.device('/gpu:0')
train_test = []
train_label_test = []
counter = 0
for i in train_data:
    if(counter==1400):
      break

    file_path = i
    image = tf.io.read_file(file_path)
    # print("image1: ", image)
    # start_time = tm.time()
    image = tf.image.decode_png(image, channels=3)
    # print("image2: ", image)
    image = tf.cast(image, tf.float32) * (1. / 255)
    # print("image3: ", image)
    # image = tf.image.convert_image_dtype(img_name, tf.float32)
    image = tf.image.resize(image, [224, 224])
    # print("image4: ", image)
    image = tf.reshape(image, (224, 224, 3))
    # print("image.shape: ", image.shape)
    train_test.append(image)
    train_label_test.append(train_labels[counter])
    counter+=1

In [10]:
tf.device('/gpu:0')
val_test = []
val_label_test = []
counter =0 
for i in val_data:
    if(counter == 600):
      break
    # counter+=1
    file_path = i
    image = tf.io.read_file(file_path)
    # print("image1: ", image)
    # start_time = tm.time()
    image = tf.image.decode_png(image, channels=3)
    # print("image2: ", image)
    image = tf.cast(image, tf.float32) * (1. / 255)
    # print("image3: ", image)
    # image = tf.image.convert_image_dtype(img_name, tf.float32)
    image = tf.image.resize(image, [224, 224])
    # print("image4: ", image)
    image = tf.reshape(image, (224, 224, 3))
    # print("image.shape: ", image.shape)
    val_test.append(image)
    val_label_test.append(val_labels[counter])
    counter+=1

In [11]:
tf.device('/gpu:0')
test_test = []
test_label_test = []
counter =0 
for i in test_data:
    if(counter == 2000):
      break
    # counter+=1
    file_path = i
    # image = tf.io.read_file(file_path)
    # # print("image1: ", image)
    # # start_time = tm.time()
    # image = tf.image.decode_png(image, channels=3)
    # # print("image2: ", image)
    # image = tf.cast(image, tf.float32) * (1. / 255)
    # # print("image3: ", image)
    # # image = tf.image.convert_image_dtype(img_name, tf.float32)
    # image = tf.image.resize(image, [224, 224])
    # # print("image4: ", image)
    # image = tf.reshape(image, (224, 224, 3))
    # print("image.shape: ", image.shape)
    test_test.append(file_path)
    test_label_test.append(test_labels[counter])
    counter+=1

In [12]:
tempData = np.array(test_test)
templabel = np.array(test_label_test)

In [13]:
file = open("/content/drive/MyDrive/Image/testData.txt", "w+")
 
# Saving the array in a text file
for item in tempData:

  content = str(item)
  file.write(content)
  content = str("\n")
  file.write(content)
file.close()
 
file = open("/content/drive/MyDrive/Image/testLabel.txt", "w+")
 
# Saving the array in a text file
for item in templabel:

  content = str(item)
  file.write(content)
  content = str("\n")
  file.write(content)
file.close()


In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

numerical_labels = le.fit_transform(train_label_test)

for label, num in zip(le.classes_, le.transform(le.classes_)):
    print(f"{label} : {num}")
train_y = np.array(numerical_labels)

numerical_labels = le.fit_transform(val_label_test)

for label, num in zip(le.classes_, le.transform(le.classes_)):
    print(f"{label} : {num}")
val_y = np.array(numerical_labels)

# numerical_labels = le.fit_transform(test_labels)

# for label, num in zip(le.classes_, le.transform(le.classes_)):
#     print(f"{label} : {num}")
# test_y = np.array(numerical_labels)

Non-Smoke : 0
Non-Smoke + Fog : 1
Smoke : 2
Smoke + Fog : 3
Non-Smoke : 0
Non-Smoke + Fog : 1
Smoke : 2
Smoke + Fog : 3


In [15]:
# function to prediction
def classify_image(image):
    image = tf.reshape(image, (1, 224, 224, 3))
    classes = [0, 1, 2, 3]
    prediction = tf.keras.Model.predict(model, image, steps=1, batch_size=1)
    return classes[np.argmax(prediction[0])]

In [16]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(4, 4)),
    Flatten(),
    Dense(units=64, activation='relu', kernel_initializer=GlorotUniform(seed=None), kernel_regularizer=l2()),
    Dropout(rate=0.2),
    Dense(units=64, activation='relu', kernel_initializer=GlorotUniform(seed=None), kernel_regularizer=l2()),
    Dropout(rate=0.2),
    Dense(units=4, activation='softmax', kernel_initializer=GlorotUniform(seed=None), kernel_regularizer=l2())
])

In [17]:
# Compile the model with an optimizer and loss function
import keras
optimizer = keras.optimizers.Adam(learning_rate=0.001)
loss_fn = keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])


In [18]:
X_train = np.array(train_test)
# X_test = np.array(test_data2)
X_val = np.array(val_test)

In [19]:
model.fit(X_train, train_y, batch_size = 16, epochs=30,  validation_data=(X_val, val_y))

Epoch 1/30
88/88 [==============================] - 14s 36ms/step - loss: 1.8676 - accuracy: 0.4921 - val_loss: 1.2546 - val_accuracy: 0.6550
Epoch 2/30
88/88 [==============================] - 2s 22ms/step - loss: 1.1294 - accuracy: 0.6793 - val_loss: 0.9575 - val_accuracy: 0.7750
Epoch 3/30
88/88 [==============================] - 2s 22ms/step - loss: 0.9843 - accuracy: 0.7293 - val_loss: 0.8311 - val_accuracy: 0.8183
Epoch 4/30
88/88 [==============================] - 2s 23ms/step - loss: 0.8596 - accuracy: 0.7814 - val_loss: 0.7286 - val_accuracy: 0.8417
Epoch 5/30
88/88 [==============================] - 2s 22ms/step - loss: 0.7540 - accuracy: 0.8179 - val_loss: 0.6316 - val_accuracy: 0.9233
Epoch 6/30
88/88 [==============================] - 2s 21ms/step - loss: 0.6777 - accuracy: 0.8557 - val_loss: 0.5919 - val_accuracy: 0.9300
Epoch 7/30
88/88 [==============================] - 2s 25ms/step - loss: 0.6567 - accuracy: 0.8700 - val_loss: 0.5124 - val_accuracy: 0.9533
Epoch 8/30
8

In [20]:
model.save("/content/drive/MyDrive/Image/Model/weight1_30epoch.h5")

****************************************************************************

In [ ]:
# train_val_data, test_data, train_val_labels, test_labels = train_test_split(frame_paths, label, test_size=0.5, random_state=42)

In [ ]:
# # save numpy array as csv file
# from numpy import asarray
# from numpy import savetxt

In [ ]:
tempData = np.array(test_data)
templabel = np.array(test_labels)

In [ ]:
# len(tempData)

In [ ]:
file = open("/content/drive/MyDrive/Image/testData.txt", "w+")
 
# Saving the array in a text file
for item in tempData:

  content = str(item)
  file.write(content)
  content = str("\n")
  file.write(content)
file.close()
 
file = open("/content/drive/MyDrive/Image/testLabel.txt", "w+")
 
# Saving the array in a text file
for item in templabel:

  content = str(item)
  file.write(content)
  content = str("\n")
  file.write(content)
file.close()

# # Displaying the contents of the text file
# file = open("/content/drive/MyDrive/Image/file3.txt", "r")
# # content = file.read()
# # print(type(content))
# Lines = file.readlines()
  
# count = 0
# # Strips the newline character
# for line in Lines:
#     count += 1
#     print(line)
#     # print("Line{}: {}".format(count, line.strip()))
 
# # print("\nContent in file1.txt:\n", content)
# file.close()

In [ ]:
# file = open("/content/drive/MyDrive/Image/file1.txt", "w+")
 
# # Saving the array in a text file
# content = str(tempData)
# file.write(content)
# file.close()
 
# # Displaying the contents of the text file
# file = open("/content/drive/MyDrive/Image/file1.txt", "r")
# content = file.read()

 
# print("\nContent in file1.txt:\n", content)
# file.close()

In [ ]:
# savetxt('/content/drive/MyDrive/Image/data.txt', test_data, delimiter=',')

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

numerical_labels = le.fit_transform(train_val_labels)

for label, num in zip(le.classes_, le.transform(le.classes_)):
    print(f"{label} : {num}")
train_val_y = np.array(numerical_labels)

numerical_labels = le.fit_transform(test_labels)

for label, num in zip(le.classes_, le.transform(le.classes_)):
    print(f"{label} : {num}")
test_y = np.array(numerical_labels)

burned-area : 0
fire-smoke : 1
fog-area : 2
green-area : 3
burned-area : 0
fire-smoke : 1
fog-area : 2
green-area : 3


#Preprocessing

In [ ]:
tf.device('/gpu:0')
train_data2 = []
for i in train_val_data:
    file_path = i
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.cast(image, tf.float32) * (1. / 255)
    image = tf.image.resize(image, [224, 224])
    image = tf.reshape(image, (224, 224, 3))
    train_data2.append(image)

In [ ]:
tf.device('/gpu:0')
test_data2 = []
for i in test_data:
    file_path = i
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.cast(image, tf.float32) * (1. / 255)
    image = tf.image.resize(image, [224, 224])
    image = tf.reshape(image, (224, 224, 3))
    test_data2.append(image)

#Function

In [ ]:
# function to prediction
def classify_image(image):
    image = tf.reshape(image, (1, 224, 224, 3))
    classes = [0, 1, 2, 3]
    prediction = tf.keras.Model.predict(model, image, steps=1, batch_size=1)
    return classes[np.argmax(prediction[0])]

#Model

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(4, 4)),
    Flatten(),
    Dense(units=64, activation='relu', kernel_initializer=GlorotUniform(seed=None), kernel_regularizer=l2()),
    Dropout(rate=0.2),
    Dense(units=64, activation='relu', kernel_initializer=GlorotUniform(seed=None), kernel_regularizer=l2()),
    Dropout(rate=0.2),
    Dense(units=4, activation='softmax', kernel_initializer=GlorotUniform(seed=None), kernel_regularizer=l2())
])

In [ ]:
# Compile the model with an optimizer and loss function
import keras
optimizer = keras.optimizers.Adam(learning_rate=0.001)
loss_fn = keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])


In [ ]:
# train_val_data, test_data, train_val_labels, test_labels = train_test_split(train_data2, train_y, test_size=0.5, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data2,train_val_y, test_size=0.3, random_state=42)

In [ ]:
from collections import Counter

In [ ]:
Counter(train_labels), Counter(val_labels), Counter(test_y)

(Counter({3: 288, 2: 416, 0: 413, 1: 521}),
 Counter({1: 230, 0: 180, 3: 123, 2: 170}),
 Counter({0: 620, 2: 483, 3: 468, 1: 771}))

In [ ]:
X_train = np.array(train_data)
X_test = np.array(test_data2)
X_val = np.array(val_data)

In [ ]:
model.fit(X_train, train_labels, batch_size = 16, epochs=30,  validation_data=(X_val, val_labels))

Epoch 1/30
103/103 [==============================] - 13s 34ms/step - loss: 1.9383 - accuracy: 0.4145 - val_loss: 1.3666 - val_accuracy: 0.5832
Epoch 2/30
103/103 [==============================] - 3s 25ms/step - loss: 1.2655 - accuracy: 0.6190 - val_loss: 1.0443 - val_accuracy: 0.7312
Epoch 3/30
103/103 [==============================] - 3s 25ms/step - loss: 1.0823 - accuracy: 0.6966 - val_loss: 0.9572 - val_accuracy: 0.7511
Epoch 4/30
103/103 [==============================] - 2s 24ms/step - loss: 0.9560 - accuracy: 0.7466 - val_loss: 0.9784 - val_accuracy: 0.7340
Epoch 5/30
103/103 [==============================] - 3s 25ms/step - loss: 0.9059 - accuracy: 0.7729 - val_loss: 0.9273 - val_accuracy: 0.7454
Epoch 6/30
103/103 [==============================] - 3s 24ms/step - loss: 0.8816 - accuracy: 0.7821 - val_loss: 0.8531 - val_accuracy: 0.7866
Epoch 7/30
103/103 [==============================] - 3s 26ms/step - loss: 0.8338 - accuracy: 0.8083 - val_loss: 0.8286 - val_accuracy: 0.806

In [ ]:
model.save("/content/drive/MyDrive/Image/Model/weight2_50epoch.h5")

In [ ]:
test_data, batch1_data, test_labels, batch1_labels = train_test_split(X_test, test_y, test_size=0.1, random_state=42)
test_data, batch2_data, test_labels, batch2_labels = train_test_split(test_data, test_labels, test_size=0.1, random_state=42)
test_data, batch3_data, test_labels, batch3_labels = train_test_split(test_data, test_labels, test_size=0.1, random_state=42)
test_data, batch4_data, test_labels, batch4_labels = train_test_split(test_data, test_labels, test_size=0.1, random_state=42)
test_data, batch5_data, test_labels, batch5_labels = train_test_split(test_data, test_labels, test_size=0.1, random_state=42)
test_data, batch6_data, test_labels, batch6_labels = train_test_split(test_data, test_labels, test_size=0.1, random_state=42)
test_data, batch7_data, test_labels, batch7_labels = train_test_split(test_data, test_labels, test_size=0.1, random_state=42)
test_data, batch8_data, test_labels, batch8_labels = train_test_split(test_data, test_labels, test_size=0.1, random_state=42)
# test_data, batch9_data, test_labels, batch9_labels = train_test_split(test_data, test_labels, test_size=0.1, random_state=42)
# test_data, batch10_data, test_labels, batch10_labels = train_test_split(test_data, test_labels, test_size=0.1, random_state=42)
batch10_data, batch9_data, batch10_labels, batch9_labels = train_test_split(test_data, test_labels, test_size=0.1, random_state=42)

In [ ]:
testData = [batch1_data, batch2_data, batch3_data, batch4_data, batch5_data ,batch6_data ,batch7_data, batch8_data, batch9_data, batch10_data]
testLabel = [batch1_labels, batch2_labels, batch3_labels, batch4_labels, batch5_labels ,batch6_labels ,batch7_labels, batch8_labels, batch9_labels, batch10_labels]

In [ ]:
yTrue = []
yPred = []
for k in range(10):
  y_true = np.array([]) 
  y_pred = np.array([])
  tbatch = testData[k]
  tlabel = testLabel[k]
  for folder in range(len(tbatch)):
      pattern = "*.jpg"
      if fnmatch.fnmatch(entry, pattern):
        predict = classify_image(tbatch[folder])
        real = tlabel[folder]
        y_true = np.append(y_true, real)
        y_pred = np.append(y_pred, predict)
  yTrue.append(y_true)
  yPred.append(y_pred)

1/1 [==============================] - 0s 39ms/step


In [ ]:
target_names = ["burned-area", "fire-smoke", "fog-area", "green-area"]
for i in range(10):
  print("Fold ",i,": ")
  # target_names = ["Non-Smoke", "Non-Smoke+Fog", "Smoke", "Smoke+Fog"]    # CCTV dataset
   # UAV dataset
  print(classification_report(yTrue[i], yPred[i], target_names=target_names))

In [ ]:
for i in range(10):
  print("Fold ",i,": ")
  hammingLoss = hamming_loss(yTrue[i], yPred[i])
  print("Hamming loss: %f" % hammingLoss)
  Accuracy = metrics.accuracy_score(yTrue[i], yPred[i])
  print("Accuracy: %f" % Accuracy)
  PrecisionMacro = metrics.precision_score(yTrue[i], yPred[i], average='macro')
  print("Precision Macro: %f" % PrecisionMacro)
  RecallMacro = metrics.recall_score(yTrue[i], yPred[i], average='macro')
  print("Recall Macro: %f" % RecallMacro)
  F1score = metrics.f1_score(yTrue[i], yPred[i], average='weighted')
  print("F1-score: %f" % F1score)

Dataset1

In [ ]:
tf.device('/gpu:0')
dir = "/content/drive/MyDrive/Image/Dataset1/"
test_dir = dir
label = []
list_of_folders = os.listdir(test_dir)
frame_paths = []

for folder in list_of_folders:
    print("folder: ", folder)
    list_of_files = os.listdir(test_dir + "/" + folder)
    pattern = "*.jpg"
    counter = 0
    for entry in list_of_files:
        if fnmatch.fnmatch(entry, pattern):
            file_path = test_dir + folder + "/" + entry
            frame_paths.append(file_path)
            label.append(folder)

folder:  Non-Smoke
folder:  Non-Smoke + Fog
folder:  Smoke
folder:  Smoke + Fog


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

numerical_labels = le.fit_transform(label)

for label, num in zip(le.classes_, le.transform(le.classes_)):
    print(f"{label} : {num}")
train_y = np.array(numerical_labels)

Non-Smoke : 0
Non-Smoke + Fog : 1
Smoke : 2
Smoke + Fog : 3


In [ ]:
len(frame_paths)

72039

In [ ]:
train_val_data, test_data, train_val_labels, test_labels = train_test_split(frame_paths, train_y, test_size=0.5, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_val_data, train_val_labels, test_size=0.3, random_state=42)

In [ ]:
tf.device('/gpu:0')
train_test = []
train_label_test = []
counter = 0
for i in train_data:
    if(counter==1400):
      break

    file_path = i
    image = tf.io.read_file(file_path)
    # print("image1: ", image)
    # start_time = tm.time()
    image = tf.image.decode_png(image, channels=3)
    # print("image2: ", image)
    image = tf.cast(image, tf.float32) * (1. / 255)
    # print("image3: ", image)
    # image = tf.image.convert_image_dtype(img_name, tf.float32)
    image = tf.image.resize(image, [224, 224])
    # print("image4: ", image)
    image = tf.reshape(image, (224, 224, 3))
    # print("image.shape: ", image.shape)
    train_test.append(image)
    train_label_test.append(train_labels[counter])
    counter+=1

In [ ]:
tf.device('/gpu:0')
test_test = []
test_label_test = []
counter =0 
for i in test_data:
    if(counter == 2000):
      break
    # counter+=1
    file_path = i
    image = tf.io.read_file(file_path)
    # print("image1: ", image)
    # start_time = tm.time()
    image = tf.image.decode_png(image, channels=3)
    # print("image2: ", image)
    image = tf.cast(image, tf.float32) * (1. / 255)
    # print("image3: ", image)
    # image = tf.image.convert_image_dtype(img_name, tf.float32)
    image = tf.image.resize(image, [224, 224])
    # print("image4: ", image)
    image = tf.reshape(image, (224, 224, 3))
    # print("image.shape: ", image.shape)
    test_test.append(image)
    test_label_test.append(test_labels[counter])
    counter+=1

In [ ]:
tf.device('/gpu:0')
val_test = []
val_label_test = []
counter =0 
for i in val_data:
    if(counter == 600):
      break
    # counter+=1
    file_path = i
    image = tf.io.read_file(file_path)
    # print("image1: ", image)
    # start_time = tm.time()
    image = tf.image.decode_png(image, channels=3)
    # print("image2: ", image)
    image = tf.cast(image, tf.float32) * (1. / 255)
    # print("image3: ", image)
    # image = tf.image.convert_image_dtype(img_name, tf.float32)
    image = tf.image.resize(image, [224, 224])
    # print("image4: ", image)
    image = tf.reshape(image, (224, 224, 3))
    # print("image.shape: ", image.shape)
    val_test.append(image)
    val_label_test.append(val_labels[counter])
    counter+=1

In [ ]:
X = np.array(train_test)
y = np.array(train_label_test)
X_val = np.array(val_test)
y_val = np.array(val_label_test)
X_test = np.array(test_test)
y_test = np.array(test_label_test)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
loss_fn = keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])


In [ ]:
model.fit(X, y, epochs=100,  validation_data=(X_val, y_val))

In [ ]:
# dir = "/home/jefferson/DOUTORADO2/DatasetDrone/test/10/" # load path with test images
dir = "/content/drive/MyDrive/Image/"
test_dir = dir + "Test2"
# listOfFolders = os.listdir(test_dir)
# print("listOfFolders: ", listOfFolders)
y_true = np.array([])  # y_true = [] list
y_pred = np.array([])
# predictTime = np.array([])
predictTime = []
cont = 0
for folder in range(len(X_test)):
    # print("folder: ", folder)
    # listOfFiles = os.listdir(test_dir + "/" + folder)
    # print("listOfFiles: ", listOfFiles)
    pattern = "*.jpg"
    # for entry in listOfFiles:
        # print("entry: ", entry)
    if fnmatch.fnmatch(entry, pattern):
      # dir2 = test_dir+"/" + folder + "/"
            # print("dir2: ", dir2)
            # start_time = time.time()
            # predict, time = classify_image(entry, dir2)
      # print("test_data[folder]: ", test_data[folder])
      predict = classify_image(X_test[folder])
            # print("predict: ", predict)
            # predictTime = np.append(predictTime, time)  # predictTime.append(time.time() - start_time)
            # predictTime.append(time)
      real = y_test[folder]
      print("predict: ", predict)
      print("real: ", real)
      # resultado = "Real: " + real + "  Predicao: " + predict
      # print(resultado)
      y_true = np.append(y_true, real)  # y_true.append(real)
      y_pred = np.append(y_pred, predict)  # y_pred.append(predict)


In [ ]:
target_names = ["Non-Smoke", "Non-Smoke+Fog", "Smoke", "Smoke+Fog"]    # CCTV dataset
# target_names = ["burned-area", "fire-smoke", "fog-area", "green-area"]   # UAV dataset
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
hammingLoss = hamming_loss(y_true, y_pred)
print("Hamming loss: %f" % hammingLoss)
Accuracy = metrics.accuracy_score(y_true, y_pred)
print("Accuracy: %f" % Accuracy)
PrecisionMacro = metrics.precision_score(y_true, y_pred, average='macro')
print("Precision Macro: %f" % PrecisionMacro)
RecallMacro = metrics.recall_score(y_true, y_pred, average='macro')
print("Recall Macro: %f" % RecallMacro)
F1score = metrics.f1_score(y_true, y_pred, average='weighted')
print("F1-score: %f" % F1score)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
loss_fn = keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=10,  validation_data=(X_val, y_val))

In [ ]:
# dir = "/home/jefferson/DOUTORADO2/DatasetDrone/test/10/" # load path with test images
dir = "/content/drive/MyDrive/Image/"
test_dir = dir + "Test2"
# listOfFolders = os.listdir(test_dir)
# print("listOfFolders: ", listOfFolders)
y_true = np.array([])  # y_true = [] list
y_pred = np.array([])
# predictTime = np.array([])
predictTime = []
cont = 0
for folder in range(len(X_test)):
    # print("folder: ", folder)
    # listOfFiles = os.listdir(test_dir + "/" + folder)
    # print("listOfFiles: ", listOfFiles)
    pattern = "*.jpg"
    # for entry in listOfFiles:
        # print("entry: ", entry)
    if fnmatch.fnmatch(entry, pattern):
      # dir2 = test_dir+"/" + folder + "/"
            # print("dir2: ", dir2)
            # start_time = time.time()
            # predict, time = classify_image(entry, dir2)
      # print("test_data[folder]: ", test_data[folder])
      predict = classify_image(X_test[folder])
            # print("predict: ", predict)
            # predictTime = np.append(predictTime, time)  # predictTime.append(time.time() - start_time)
            # predictTime.append(time)
      real = y_test[folder]
      print("predict: ", predict)
      print("real: ", real)
      # resultado = "Real: " + real + "  Predicao: " + predict
      # print(resultado)
      y_true = np.append(y_true, real)  # y_true.append(real)
      y_pred = np.append(y_pred, predict)  # y_pred.append(predict)


In [ ]:
target_names = ["Non-Smoke", "Non-Smoke+Fog", "Smoke", "Smoke+Fog"]    # CCTV dataset
# target_names = ["burned-area", "fire-smoke", "fog-area", "green-area"]   # UAV dataset
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
hammingLoss = hamming_loss(y_true, y_pred)
print("Hamming loss: %f" % hammingLoss)
Accuracy = metrics.accuracy_score(y_true, y_pred)
print("Accuracy: %f" % Accuracy)
PrecisionMacro = metrics.precision_score(y_true, y_pred, average='macro')
print("Precision Macro: %f" % PrecisionMacro)
RecallMacro = metrics.recall_score(y_true, y_pred, average='macro')
print("Recall Macro: %f" % RecallMacro)
F1score = metrics.f1_score(y_true, y_pred, average='weighted')
print("F1-score: %f" % F1score)